In [1]:
import sqlite3
import csv
import pandas as pd
import traceback
import sys
from datetime import datetime
import matplotlib.pyplot as plt

# Challenge 1

In [2]:
db = "challenge.db"

## Creacion de Tablas

In [3]:
## CREACION DE TABLAS

con = sqlite3.connect(db)
cur = con.cursor()


try:
    sql_file = open("createTables.sql")
    sql_as_string = sql_file.read()
    cur.executescript(sql_as_string)
except sqlite3.Error as er:
     print('SQLite error: %s' % (' '.join(er.args)))
     print("Exception class is: ", er.__class__)
     print('SQLite traceback: ')
     exc_type, exc_value, exc_tb = sys.exc_info()
     print(traceback.format_exception(exc_type, exc_value, exc_tb))
     con.close()

con.close()

## Migrar datos desde archivos CSV

Leer los CVS

In [4]:
depts = pd.read_csv('data/departments.csv', header=0)
emps = pd.read_csv('data/hired_employees.csv')
jobs = pd.read_csv('data/jobs.csv')

Generar Dataframes

In [5]:
depts = pd.read_csv('data/departments.csv', header=None)
depts.columns = ['id', 'department']

jobs = pd.read_csv('data/jobs.csv', header=None)
jobs.columns = ['id', 'job']

hired_employees = pd.read_csv('data/hired_employees.csv', header=None)
hired_employees.columns = ['id', 'name', 'datetime', 'department_id', 'job_id']


Llenar las tablas a partir de los DataFrames

In [6]:

con = sqlite3.connect(db)

try:
    depts.to_sql('departments', con, if_exists='replace', index=False)
except sqlite3.Error as er:
     print('SQLite error: %s' % (' '.join(er.args)))
     print("Exception class is: ", er.__class__)
     print('SQLite traceback: ')
     exc_type, exc_value, exc_tb = sys.exc_info()
     print(traceback.format_exception(exc_type, exc_value, exc_tb))
     
try:
    jobs.to_sql('jobs', con, if_exists='replace', index=False)
except sqlite3.Error as er:
     print('SQLite error: %s' % (' '.join(er.args)))
     print("Exception class is: ", er.__class__)
     print('SQLite traceback: ')
     exc_type, exc_value, exc_tb = sys.exc_info()
     print(traceback.format_exception(exc_type, exc_value, exc_tb))
     
try:
    hired_employees.to_sql('hired_employees', con, if_exists='replace', index=False)
except sqlite3.Error as er:
     print('SQLite error: %s' % (' '.join(er.args)))
     print("Exception class is: ", er.__class__)
     print('SQLite traceback: ')
     exc_type, exc_value, exc_tb = sys.exc_info()
     print(traceback.format_exception(exc_type, exc_value, exc_tb))
     
con.close()

## Testing cargando DataFrames desde la Base

In [7]:
depts = None
jobs = None
hired_employees = None

con = sqlite3.connect(db)

depts = pd.read_sql('SELECT * FROM DEPARTMENTS', con)
jobs = pd.read_sql('SELECT * FROM JOBS', con)
hired_employees = pd.read_sql('SELECT * FROM HIRED_EMPLOYEES', con)

con.close()